In [42]:
import conllu, os
import xml.etree.ElementTree as et
import pandas as pd

from ufal.udpipe import Model, Pipeline
from gensim.models import KeyedVectors
from collections import defaultdict

In [35]:
xtree = et.parse("development/SentiRuEval_rest_train.xml")
root = xtree.getroot()
data = []
columns = ['id', 'food', 'service', 'text']
for review in root:
    text_id = int(review.attrib['id'])
    
    scores = review.find('scores')
    
    food = int(scores.find('food').text)
    service = int(scores.find('service').text)
    
    text = review.find('text').text
    
    data.append({'id': text_id,
                'food': food,
                'service': service,
                'text': text})
    

df = pd.DataFrame(data, columns=columns)
df = df.set_index('id')

In [36]:
udpipe_model = Model.load('udpipe_models/russian-syntagrus-ud-2.0-170801.udpipe')
pipeline = Pipeline(udpipe_model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

In [37]:
def udpipe_lemmatize(text, ud_pipeline=pipeline):
    processed = conllu.parse(pipeline.process(text))
    return [[token['lemma'] for token in sent] for sent in processed]

In [38]:
food_words = dict()
service_words = dict()

with open('development/Food_words.txt', 'r', encoding='utf-8') as inp:
    for line in inp.readlines():
        aspect, word, score = line.strip().split('        ')
        food_words[word] = score

with open('development/Service_words.txt', 'r', encoding='utf-8') as inp:
    for line in inp.readlines():
        aspect, word, score = line.strip().split('\t')
        service_words[word] = score

In [39]:
new_food_words = dict()
new_service_words = dict()

for file in os.listdir('разметка_финал'):
    if file.endswith('.tsv'):
        print(file)
        
        with open(os.path.join('разметка_финал',file), 'r', encoding='utf-8') as inp:
            lines = inp.readlines()

        conllu_path = os.path.join('conllu_data', file)
        with open(conllu_path, 'r', encoding='utf-8') as inp:
            conll = conllu.parse(inp.read())

        for line in lines:
            if line:
                sent_id, token_ids, aspect, mark = line.strip().split('\t')
                mark = int(mark)
                sent_id = int(sent_id)-1
                token_ids = [int(i) for i in token_ids.split(',')]
                start_id, end_id = int(token_ids[0])-1, int(token_ids[-1])
                try:
                    tokens = ' '.join(token['lemma'] for token in conll[sent_id][start_id:end_id])
                except:
                    print(sent_id, start_id, end_id)
                if aspect.strip() == 'Service' and tokens not in new_service_words:
                    new_service_words[tokens] = mark
                elif aspect.strip() == 'Food' and tokens not in new_food_words:
                    new_food_words[tokens] = mark

12943.tsv
13823.tsv
20086.tsv
28083.tsv
32840.tsv
32856.tsv
33591.tsv
33693.tsv
35486.tsv
5648.tsv


Посмотрим на пересечения выделенных нами словарей:

In [40]:
set(food_words) & set(new_food_words), set(service_words) & set(new_service_words),

({'большой',
  'вкусный',
  'невкусный',
  'отличный',
  'понравиться',
  'прекрасный',
  'сытный'},
 {'вежливый', 'ненавязчивый', 'приятный', 'хамоватый', 'хороший'})

И на их объединение:

In [41]:
set(food_words)|set(new_food_words), set(service_words)|set(new_service_words)

({'10 балл',
  'большой',
  'великолепный',
  'весь остыть',
  'вкусно',
  'вкусный',
  'впечатлять',
  'высокий все похвасть',
  'горячий',
  'достойный',
  'единственный',
  'интересный',
  'не впечатлять',
  'не очень дорого',
  'невкусный',
  'нежный',
  'необычный',
  'отличный',
  'очень большой',
  'очень вкусный',
  'плохой',
  'понравиться',
  'посредственно',
  'прекрасный',
  'приятный',
  'различный',
  'разнообразный',
  'разнообразный вкусный',
  'расстроить',
  'свежий',
  'совершенно отвратительный',
  'странный',
  'сытный',
  'хороший'},
 {'вежливый',
  'веселый',
  'внимательный',
  'вполне приемлимый',
  'высокий качество',
  'гостеприимный',
  'доброжелательный',
  'дружелюбный',
  'душевный',
  'качественный',
  'красивый',
  'милый',
  'не слишком вежливо',
  'недолгий',
  'ненавязчивый',
  'оперативность',
  'оперативный',
  'отзывчивый',
  'отличный',
  'очень аккуратно',
  'очень приветливый',
  'плохо знать меню',
  'понравиться',
  'приветливый',
  'приятный

Извлечём 1,2,3-граммы из development-корпуса:

In [46]:
def extract_123grams(texts, min_freq=5):
    ugram_freq_dict, bigram_freq_dict, trigram_freq_dict = defaultdict(int), defaultdict(int), defaultdict(int)
    print("Lemmatizing with udpipe")
    texts = udpipe_lemmatize('\n\n'.join(texts))
    print("Extracting 1,2,3 grams")
    ## extract 1grams
    for sent in texts:
        for token1, token2, token3 in zip(sent, sent[1:], sent[2:]):
            ugram_freq_dict[token1] += 1
            bigram_freq_dict[token1+' '+token] += 1
            trigram_freq_dict[token1+' '+token2+' '+token3] += 1
        
        if len(sent) > 0:
            ugram_freq_dict[sent[-1]] += 1
            if len(sent) > 1:
                bigram_freq_dict[sent[-2]+' '+sent[-1]] += 1
                ugram_freq_dict[sent[-2]] += 1
    
    ugram_freq_dict = {k:v for k,v in ugram_freq_dict.items() if v > min_freq}
    bigram_freq_dict = {k:v for k,v in bigram_freq_dict.items() if v > min_freq}
    trigram_freq_dict = {k:v for k,v in trigram_freq_dict.items() if v > min_freq}
    
    return ugram_freq_dict, bigram_freq_dict, trigram_freq_dict

In [ ]:
ugrams, bigrams, trigrams = extract_123grams(df['text'])